# Gridsearch over hyperparameters of MNIST autoencoder with sparsity and participation constraints

In this notebook, we write a tensorflow wrapper for sklearn, such that we can use the sklearn gridsearch framework to tune the hyperparameters of our network.

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(42)

import numpy as np
import pickle

from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_X_y, check_array
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

## Download and read mnist data
mnist = input_data.read_data_sets('mnist_data', one_hot=True)

### Helper functions
def sshift(c, k=100.):
    """How to shift the sigmoid along the x-axis, such that sigmoid(0) = c.

    Args:
        c: y-value of sigmoid(0).
        k: Stretching factor of sigmoid.

    Returns:
        Returns x_s such that sigmoid(x-x_s) = c for x = 0. Note, that the original sigmoid has the property that
        sigmoid(x) = 0.5 for x = 0.
    """
    return 1./k * tf.log((1.-c)/c)

def sigmoid(x, x_s, k=100.):
    """Compute a sigmoid fuction.

    Args:
        x: Input to sigmoid
        c: y-value of sigmoid(0).
        k: Stretching factor of sigmoid.

    Returns:
        Returns 1/(1+exp(-k*(x-x_s)))
    """
    return tf.div(1.,tf.add(1.,tf.exp(-k * (x-x_s))))

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz


In [2]:
class AutoencoderMNIST(BaseEstimator):
    """ A class that adheres the scikit-learn estimator API, 
    such that GridSearch can be used.
    
    Attributes:
        See parameters of constructor! Additional attributes are:
        
        size_input: Number of neurons in input and output layer.
        inputs: The input tensor (containing an input batch).
        activity_trace_h[1,2,3]: The smoothed activation trace of hidden units.
        h[1,2,3]: Tensor, containing the hidden activity.
        output: Output tensor.
        loss: The loss tensor (a scalar, that measures performance on current input).
        sess: The tensorflow session.
    """
    def __init__(self, learning_rate=1.0, alpha_s=0.5, alpha_p=0.5, alpha_b=0.5, sparsity=0.5, \
                 gamma=0.99, size_h1=200, size_h2=30, batch_size=32, num_epochs=10000):
        """Construct the neural network using the given parameters.
        
        Args:
            learning_rate: Initial learning rate, which is halved in every evaluation 
                epoch.
            alpha_s: Trade-off parameter for sparsity regularizer.
            alpha_p: Trade-off parameter for participation regularizer.
            alpha_b: Trade-off parameter for binary activation regularizer.
            sparsity: Sparsity level that should be enforced within a population (layer).
            gamma: Parameter to control exponential smoothing of activation traces.
            size_h1: Number of neurons in first hidden layer.
            size_h2: Number of neurons in second hidden layer (middle layer).
            batch_size: Size of mini batches.
            num_epochs: Number of training epochs.
        """
        self.learning_rate = learning_rate
        self.alpha_s = alpha_s
        self.alpha_p = alpha_p
        self.alpha_b = alpha_b
        self.sparsity = sparsity
        self.gamma = gamma
        self.size_input = 784
        self.size_h1 = size_h1
        self.size_h2 = size_h2
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        
    def _build_network(self):
        alpha_s = self.alpha_s
        alpha_p = self.alpha_p
        alpha_b = self.alpha_b
        sparsity = self.sparsity
        participation = (1-sparsity)
        gamma = self.gamma
        
        tf.reset_default_graph()
        
        ## Build Network.
        
        # Network Inputs. Note that activity traces are maintained outside of the scope
        # of backprop and therefore have to be network inputs.
        self.inputs = tf.placeholder(tf.float32, shape=[None, self.size_input])
        self._lambda = tf.placeholder(tf.float32, shape=[]) # Learning rate

        self.activity_trace_h1 = tf.placeholder(tf.float32, shape=[self.size_h1])
        self.activity_trace_h2 = tf.placeholder(tf.float32, shape=[self.size_h2])
        self.activity_trace_h3 = tf.placeholder(tf.float32, shape=[self.size_h1])


        # Create a five layer fully-connected network:
        # size_input -> size_h1 -> size_h2 -> size_h1 -> size_input
        # We use sigmoid units, as they are more similar to prob. binomial 
        # neurons as e.g. ReLUs.
        self.h1 = tf.contrib.layers.fully_connected(self.inputs, \
            self.size_h1, activation_fn=tf.nn.sigmoid)
        self.h2 = tf.contrib.layers.fully_connected(self.h1, \
            self.size_h2, activation_fn=tf.nn.sigmoid)
        self.h3 = tf.contrib.layers.fully_connected(self.h2, \
            self.size_h1, activation_fn=tf.nn.sigmoid)
        self.outputs = tf.contrib.layers.fully_connected(self.h3, \
            self.size_input, activation_fn=tf.nn.sigmoid)

        ## Compute Loss.
        # Reconstruction Error
        # As the other loss terms (e.g. sparsity) are normalized to layer sizes, 
        # we should do this here too.
        # Note, that the function L2 loss doesn't use the sqrt op.
        self._rec_err = tf.nn.l2_loss(self.outputs-self.inputs) / float(self.size_input)

        # Add sparsity within layer to loss
        # (We only consider hidden layers, as the other layer shall be fully 
        # governed by the input)
        self._loss_s = 0
        # Compute binary loss.
        self._loss_b = 0
        for layer in [self.h1,self.h2,self.h3]:
            n_l = float(int(layer.shape[1]))
            self._loss_s += tf.reduce_mean(tf.reduce_sum(layer, 1)/n_l - participation)**2
            
            self._loss_b += tf.reduce_mean(tf.pow(layer,2)*tf.pow(layer-1,2))

        # Add participation loss for all neurons
        self._loss_p = 0
        activity_traces = [self.activity_trace_h1, self.activity_trace_h2, \
                           self.activity_trace_h3]
        x_s = sshift(participation) # sigmoid shift
        for ii, layer in enumerate([self.h1,self.h2,self.h3]):
            n_l = float(int(layer.shape[1]))
            a_trace = activity_traces[ii]

            ## Simple loss (unstable minima)
            #self._loss_p += tf.reduce_mean(tf.reduce_sum( \
            #    (gamma*a_trace + (1-gamma)*layer - participation)**2, 1)/n_l)

            ## Loss with minimas relatively stable around 0 or 1 
            ## (depending on trace error)
            # Root of the simple loss formulation.
            root = - 1/(1-gamma) * (gamma*a_trace - participation)
            sigval = sigmoid(participation - a_trace, x_s)
            self._loss_p += tf.reduce_mean(tf.reduce_sum( \
                (gamma*a_trace + (1-gamma)*layer - participation + \
                 (1-gamma) * (root - sigval))**2, 1)/n_l)

        self.loss = self._rec_err + alpha_s * self._loss_s + alpha_p * self._loss_p + \
            alpha_b * self._loss_b
        
        return self
        
    def fit(self, X, y):
        """ Train the model implemented by this class.
        
        NOTE, the input parameters are ignored. They just remain to ensure 
        compatibility with the sklearn framework. Instead, the MNIST training
        set is used.
        
        Args:
            X: array-like or sparse matrix of shape = [n_samples, n_features]
                The training input samples.
            y: array-like, shape = [n_samples] or [n_samples, n_outputs]
                The target values (class labels in classification, real numbers in
                regression).
                
        Returns:
            self
        """
        #X, y = check_X_y(X, y)
        
        self._build_network()
        
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())

        train_step = tf.train.GradientDescentOptimizer( \
            learning_rate=self.learning_rate).minimize(self.loss)

        # Initial activation traces.
        at_h1 = np.ones(self.size_h1) * (1-self.sparsity)
        at_h2 = np.ones(self.size_h2) * (1-self.sparsity)
        at_h3 = np.ones(self.size_h1) * (1-self.sparsity)

        for i in range(self.num_epochs):
            batch = mnist.train.next_batch(self.batch_size)

            # Decay learning rate.
            if i % 1000 == 0:
                self.learning_rate /= 2

            # Run backprop
            train_step.run( \
                feed_dict={self.inputs: batch[0], self._lambda: self.learning_rate, \
                           self.activity_trace_h1: at_h1, self.activity_trace_h2: at_h2, \
                           self.activity_trace_h3: at_h3})

            # Update activity traces
            activity_h1, activity_h2, activity_h3 = self.sess.run( \
                [self.h1, self.h2, self.h3], feed_dict={self.inputs: batch[0], \
                self._lambda: self.learning_rate, self.activity_trace_h1: at_h1, \
                self.activity_trace_h2: at_h2, self.activity_trace_h3: at_h3})
            # We update the traces by the mean activity achieved in the current batch
            activity_h1 = np.mean(activity_h1, axis=0)
            activity_h2 = np.mean(activity_h2, axis=0)
            activity_h3 = np.mean(activity_h3, axis=0)

            gamma = self.gamma
            at_h1 = gamma * at_h1 + (1-gamma) * activity_h1
            at_h2 = gamma * at_h2 + (1-gamma) * activity_h2
            at_h3 = gamma * at_h3 + (1-gamma) * activity_h3
    
        self._at_h1 = at_h1
        self._at_h2 = at_h2
        self._at_h3 = at_h3
    
        # Return the estimator
        return self

    def predict(self, X):
        """ This function implements no logic, as it is simply reqoired by the estimator template.
        
        Args:
            X: array-like or sparse matrix of shape = [n_samples, n_features]
                The training input samples.
                
        Returns:
            y: array-like, shape = [n_samples] or [n_samples, n_outputs]
                The target values (class labels in classification, real numbers in
                regression).
        """
        raise
        X = check_array(X)
        return X[:, 0]**2
    
    def score(self, X, y=None):
        """ Compute the loss on the full test set.
        
        NOTE, the input parameters are ignored. They just remain to ensure 
        compatibility with the sklearn framework. Instead, the MNIST test
        set is used.
        
        Args:
            X: array-like or sparse matrix of shape = [n_samples, n_features]
                The training input samples.
            y: array-like, shape = [n_samples] or [n_samples, n_outputs]
                The target values (class labels in classification, real numbers in
                regression).
                
        Returns:
            The loss computed over the complete test set.
        """
        test_images = mnist.test.images
        loss = self.sess.run(self.loss, feed_dict={
            self.inputs: test_images, self._lambda: self.learning_rate, \
           self.activity_trace_h1: self._at_h1, self.activity_trace_h2: self._at_h2, \
           self.activity_trace_h3: self._at_h3})
        
        return loss

## Defining the parameter grid

Here we initiate the actual gridsearch and define the parameters through which we sweep. We then output a sorted list of tuples (params, loss). Loss is the loss value achieved in one training (we start one training per point in our parameter grid), and by which the list is sorted. Params are the hyerparameters used for this particular model training.

In [ ]:
parameters = {
    'learning_rate':[0.1,1.0,10,100],
    'alpha_s':[0.5,1.0,2.0,5.0,10.0,100.0],
    'alpha_p':[0.5,1.0,2.0,5.0,10.0,100.0],
    'alpha_b':[0.5,1.0,2.0,5.0,10.0,100.0],
    'sparsity':[0.25,0.33,0.5,0.66,0.75],
    'gamma':[0.99],
    'size_h1':[200],
    'size_h2':[30,45,60],
    'batch_size':[32],
    'num_epochs':[10000]
}

ae = AutoencoderMNIST()
# Note, that tensorflow gpu version might cause trouble when using multithreading.
# Note, we are not using cross-validation.
clf = GridSearchCV(ae, parameters, cv=ShuffleSplit(test_size=0.20, n_splits=1, random_state=0), \
                  n_jobs=-1, verbose=1)
clf.fit([0,0], [0,0])

results = list(zip(clf.cv_results_['params'], clf.cv_results_['mean_test_score']))
results = sorted(results, key=lambda x: x[1])
results

## Save results to file

In [ ]:
pickle.dump( clf.cv_results_, open( "gridsearch_results_11-19-17.p", "wb" ) )

## Read and analyse results

In [8]:
with open("gridsearch_results_11-19-17.p", "rb") as f:
    data = pickle.load(f)
    
results = list(zip(data['params'], data['mean_test_score']))
results = sorted(results, key=lambda x: x[1])

num = 0
for i, r in enumerate(results):
    if r[0]['size_h2'] == 30:
        print("Rank: %d" % i)
        print(r)
        num += 1
        if num == 10:
            break

Rank: 33
({'alpha_b': 0.5, 'alpha_p': 100.0, 'alpha_s': 0.5, 'batch_size': 32, 'gamma': 0.99, 'learning_rate': 10, 'num_epochs': 10000, 'size_h1': 200, 'size_h2': 30, 'sparsity': 0.75}, 90.057632446289062)
Rank: 35
({'alpha_b': 0.5, 'alpha_p': 100.0, 'alpha_s': 1.0, 'batch_size': 32, 'gamma': 0.99, 'learning_rate': 10, 'num_epochs': 10000, 'size_h1': 200, 'size_h2': 30, 'sparsity': 0.75}, 90.190444946289062)
Rank: 51
({'alpha_b': 0.5, 'alpha_p': 100.0, 'alpha_s': 0.5, 'batch_size': 32, 'gamma': 0.99, 'learning_rate': 10, 'num_epochs': 10000, 'size_h1': 200, 'size_h2': 30, 'sparsity': 0.66}, 92.385345458984375)
Rank: 60
({'alpha_b': 0.5, 'alpha_p': 100.0, 'alpha_s': 2.0, 'batch_size': 32, 'gamma': 0.99, 'learning_rate': 10, 'num_epochs': 10000, 'size_h1': 200, 'size_h2': 30, 'sparsity': 0.75}, 94.083953857421875)
Rank: 68
({'alpha_b': 0.5, 'alpha_p': 100.0, 'alpha_s': 1.0, 'batch_size': 32, 'gamma': 0.99, 'learning_rate': 10, 'num_epochs': 10000, 'size_h1': 200, 'size_h2': 30, 'sparsity

In [9]:
num = 0
for i, r in enumerate(results):
    if r[0]['size_h2'] == 45:
        print("Rank: %d" % i)
        print(r)
        num += 1
        if num == 10:
            break

Rank: 6
({'alpha_b': 0.5, 'alpha_p': 100.0, 'alpha_s': 0.5, 'batch_size': 32, 'gamma': 0.99, 'learning_rate': 10, 'num_epochs': 10000, 'size_h1': 200, 'size_h2': 45, 'sparsity': 0.75}, 80.040061950683594)
Rank: 9
({'alpha_b': 0.5, 'alpha_p': 100.0, 'alpha_s': 0.5, 'batch_size': 32, 'gamma': 0.99, 'learning_rate': 10, 'num_epochs': 10000, 'size_h1': 200, 'size_h2': 45, 'sparsity': 0.66}, 80.479019165039062)
Rank: 21
({'alpha_b': 0.5, 'alpha_p': 100.0, 'alpha_s': 1.0, 'batch_size': 32, 'gamma': 0.99, 'learning_rate': 10, 'num_epochs': 10000, 'size_h1': 200, 'size_h2': 45, 'sparsity': 0.75}, 85.653648376464844)
Rank: 25
({'alpha_b': 0.5, 'alpha_p': 100.0, 'alpha_s': 5.0, 'batch_size': 32, 'gamma': 0.99, 'learning_rate': 10, 'num_epochs': 10000, 'size_h1': 200, 'size_h2': 45, 'sparsity': 0.75}, 86.641769409179688)
Rank: 26
({'alpha_b': 0.5, 'alpha_p': 100.0, 'alpha_s': 1.0, 'batch_size': 32, 'gamma': 0.99, 'learning_rate': 10, 'num_epochs': 10000, 'size_h1': 200, 'size_h2': 45, 'sparsity':

In [10]:
results[:40]

[({'alpha_b': 0.5,
   'alpha_p': 100.0,
   'alpha_s': 0.5,
   'batch_size': 32,
   'gamma': 0.99,
   'learning_rate': 10,
   'num_epochs': 10000,
   'size_h1': 200,
   'size_h2': 60,
   'sparsity': 0.75},
  75.356842041015625),
 ({'alpha_b': 0.5,
   'alpha_p': 100.0,
   'alpha_s': 5.0,
   'batch_size': 32,
   'gamma': 0.99,
   'learning_rate': 10,
   'num_epochs': 10000,
   'size_h1': 200,
   'size_h2': 60,
   'sparsity': 0.66},
  75.595344543457031),
 ({'alpha_b': 0.5,
   'alpha_p': 100.0,
   'alpha_s': 2.0,
   'batch_size': 32,
   'gamma': 0.99,
   'learning_rate': 10,
   'num_epochs': 10000,
   'size_h1': 200,
   'size_h2': 60,
   'sparsity': 0.75},
  77.370628356933594),
 ({'alpha_b': 0.5,
   'alpha_p': 100.0,
   'alpha_s': 0.5,
   'batch_size': 32,
   'gamma': 0.99,
   'learning_rate': 10,
   'num_epochs': 10000,
   'size_h1': 200,
   'size_h2': 60,
   'sparsity': 0.66},
  77.700645446777344),
 ({'alpha_b': 0.5,
   'alpha_p': 100.0,
   'alpha_s': 5.0,
   'batch_size': 32,
   'gamm